In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import posixpath

import wfdb

from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt


In [ ]:
S1_S1=pd.read_csv('/kaggle/input/raw-data-liewaves/S10S1.csv')
S1_S1

In [ ]:
S1_S2=pd.read_csv('/kaggle/input/raw-data-liewaves/S10S2.csv')
S1_S2

In [ ]:
plt.figure(figsize=(24,8))

# Define the offset
offset = 500  # adjust this value as needed

# Plot each channel with an increasing offset
channels = ['EEG.AF3', 'EEG.T7', 'EEG.Pz','EEG.T8','EEG.AF4']  # replace with your actual channel names
for i, channel in enumerate(channels):
    plt.plot(S1_S1[channel] + i * offset, label=channel)

plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.figure(figsize=(24,8))

# Define the offset
offset = 500  # adjust this value as needed

# Plot each channel with an increasing offset
channels = ['EEG.AF3', 'EEG.T7', 'EEG.Pz','EEG.T8','EEG.AF4']  # replace with your actual channel names
for i, channel in enumerate(channels):
    plt.plot(S1_S2[channel] + i * offset, label=channel)

plt.legend(loc='upper right')
plt.show()

In [ ]:
import pandas as pd

# Load normal and abnormal EEG data from CSV files
normal_data = pd.read_csv('/kaggle/input/raw-data-liewaves/S10S1.csv')  # tell the truth
abnormal_data = pd.read_csv('/kaggle/input/raw-data-liewaves/S10S2.csv') # lie

# Add a label column to distinguish between normal (0) and abnormal (1) signals
normal_data['Label'] = 0
abnormal_data['Label'] = 1

# Concatenate normal and abnormal data
merged_data = pd.concat([normal_data, abnormal_data], ignore_index=True)

# Shuffle the merged data
shuffled_data = merged_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert shuffled_data to DataFrame explicitly
shuffled_data_df = pd.DataFrame(shuffled_data)

# Save the shuffled merged data to a new CSV file
shuffled_data_df.to_csv('merged_signals.csv', index=False)

# Display the first few rows of the shuffled merged data
print(shuffled_data_df.head())


In [ ]:
shuffled_data

# splitting the data

In [ ]:
import numpy as np

# Generate synthetic EEG data for demonstration
# Assume 3 channels and 1000 data points per channel
num_channels = 6
num_data_points = 19200
shuffled_data = np.random.randn(num_channels, num_data_points)

# Split data into training and test sets
split_ratio = 0.7  # 70% for training, 30% for testing

# Calculate the number of samples for training and test sets
num_train_samples = int(num_data_points * split_ratio)
num_test_samples = num_data_points - num_train_samples

# Split data
train_data = shuffled_data[:, :num_train_samples]
test_data = shuffled_data[:, num_train_samples:]

# Optionally, if you also want to split the filtered data, you can do it as follows:
# train_filtered_data = filtered_data[:, :num_train_samples]
# test_filtered_data = filtered_data[:, num_train_samples:]

# Print the shapes of the training and test data sets
print(f"Shape of train_data: {train_data.shape}")
print(f"Shape of test_data: {test_data.shape}")


In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist_freq = 0.5 * fs
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Generate synthetic EEG data for demonstration
num_channels = 5
num_data_points = 19200
fs = 128  # Sampling frequency in Hz
t = np.linspace(0, 1, num_data_points, endpoint=False)
shuffled_data = np.random.randn(num_channels, num_data_points)

# Apply bandpass filter to each channel
lowcut = 0.5  # Lower cutoff frequency in Hz
highcut = 45  # Higher cutoff frequency in Hz
filtered_data = np.zeros_like(shuffled_data)

for i in range(num_channels):
    filtered_data[i, :] = butter_bandpass_filter(shuffled_data[i, :], lowcut, highcut, fs)

# Plot original and filtered EEG signals for all channels in separate subplots
fig, axs = plt.subplots(num_channels, 1, figsize=(12, 6*num_channels))

for i in range(num_channels):
    axs[i].plot(t, shuffled_data[i, :], label=f'Original EEG (Channel {i+1})', alpha=0.7)
    axs[i].plot(t, filtered_data[i, :], label=f'Filtered EEG (Channel {i+1})', alpha=0.7)
    axs[i].set_title(f'Original and Filtered EEG Signals (Channel {i+1})')
    axs[i].set_xlabel('Time (s)')
    axs[i].set_ylabel('Amplitude')
    axs[i].grid(True)
    axs[i].legend()

plt.tight_layout()
plt.show()





In [ ]:
filtered_data

# converting filterd data to CSV file

In [ ]:
import pandas as pd
import numpy as np

# Load existing CSV file to get column names
shuffled_data = pd.read_csv('merged_signals.csv')

# Get column names from existing DataFrame
column_names = shuffled_data.columns.tolist()[:5]

# Assuming filtered_data is a NumPy array with shape (19200, 5)
# Transpose filtered_data to have 5 columns and 19200 rows
filtered_data_transposed = filtered_data.T

# Convert transposed array to DataFrame with existing column names
df = pd.DataFrame(filtered_data_transposed, columns=column_names)

# Save DataFrame to CSV file
df.to_csv('array_data.csv', index=False)


In [ ]:
df

# filtered data with bandpass

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('array_data.csv')

# Get column names (assuming each column represents a different channel)
channels = df.columns

# Create a new figure
plt.figure(figsize=(15, 10))

# Plot each channel in a separate subplot
for i, channel in enumerate(channels):
    plt.subplot(len(channels), 1, i+1)
    plt.plot(cleaned_data[channel])
    plt.title(f'Channel {channel}')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


# individual channels ATAR algorithm

In [ ]:
import mne
import pandas as pd

# Load EEG data from CSV
df = pd.read_csv('array_data.csv')

# Iterate over each EEG channel for individual processing
for channel_name in df.columns[1:]:
    # Create MNE Raw object from the channel data
    info = mne.create_info(ch_names=[channel_name], sfreq=250, ch_types='eeg')
    raw_channel = mne.io.RawArray(df[channel_name].values.reshape(1, -1), info)

    # Apply preprocessing steps (e.g., filtering, artifact removal) to the channel data
    # Example: Bandpass filter
    raw_channel.filter(l_freq=0.5, h_freq=40)

    # Save cleaned channel data back to the DataFrame
    df[channel_name] = raw_channel.get_data()[0]

# Save cleaned EEG data with individual channel processing to CSV
df.to_csv('cleaned_eeg_data.csv', index=False)

In [ ]:
import pandas as pd

# Read the CSV file
cleaned_data = pd.read_csv('cleaned_eeg_data.csv')

# Display the first few rows of the DataFrame
print(cleaned_data.head())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
cleaned_data = pd.read_csv('cleaned_eeg_data.csv')

# Get column names (assuming each column represents a different channel)
channels = cleaned_data.columns

# Create a new figure
plt.figure(figsize=(15, 10))

# Plot each channel in a separate subplot
for i, channel in enumerate(channels):
    plt.subplot(len(channels), 1, i+1)
    plt.plot(cleaned_data[channel])
    plt.title(f'Channel {channel}')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the first CSV file
df = pd.read_csv('array_data.csv')

# Read the second CSV file for comparison
cleaned_data = pd.read_csv('cleaned_eeg_data.csv')


# Get column names (assuming each column represents a different channel)
channels = df.columns

# Create a new figure
plt.figure(figsize=(15, 10))

# Plot each channel in a separate subplot
for i, channel in enumerate(channels):
    plt.subplot(len(channels), 1, i+1)
    plt.plot(df[channel], label=f'filtered data with bandpass (Channel {channel})', alpha=0.7)
    plt.plot(cleaned_data[channel], label=f'cleaned data with ATAR (Channel {channel})', alpha=0.7)
    plt.title(f'Channel {channel}')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.grid(True)
    plt.legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()


# collective channels ATAR algorithm

In [ ]:
import mne
import pandas as pd

# Load EEG data from CSV
df = pd.read_csv('array_data.csv')

# Create MNE Raw object from the DataFrame
info = mne.create_info(ch_names=df.columns[1:], sfreq=250, ch_types='eeg')
raw = mne.io.RawArray(df.values[:, 1:].T, info)

# Perform collective channel processing (e.g., ICA)
ica = mne.preprocessing.ICA(n_components=20, random_state=42)
ica.fit(raw)

# Plot ICA components to identify artifacts
ica.plot_components()

# Choose components to exclude based on artifact patterns
ica.exclude = [0, 1, 2]  # Example: components to exclude

# Apply artifact removal to all channels
df = raw.copy().apply_ica(ica)

# Save cleaned EEG data to CSV
cleaned_eeg_data2 = pd.DataFrame(df.get_data().T, columns=df.columns[1:])
cleaned_eeg_data2.to_csv('cleaned_eeg_data2.csv', index=False)